# Make DEMS
Loop through each DEM tile, find the necessary LAS files, 

In [ ]:
#!pip install whitebox

In [11]:
import os
from shapely.geometry import box
import geopandas as gpd
from whitebox import WhiteboxTools

DEM_TINDEX = '/home/jovyan/DEM_tindex.gpkg'
LAS_TINDEX = '/home/jovyan/LAS_tindex.gpkg'

In [9]:
dems = gpd.read_file(DEM_TINDEX)

east = dems[dems['zone'] == 'east'].to_crs({'init': 'epsg:3443'})
west = dems[dems['zone'] == 'west'].to_crs({'init': 'epsg:3444'})
print(f'{len(east)} east tiles, {len(west)} west tiles')

516 east tiles, 572 west tiles


,srs,zone,name,geometry
0,EPSG:3443,east,east_719_178,POLYGON ((-89.29442519763219 41.56450650515664...
1,EPSG:3443,east,east_719_174,POLYGON ((-89.29276436950822 41.45210485851405...
2,EPSG:3443,east,east_719_170,POLYGON ((-89.29111293732947 41.33970092370969...
3,EPSG:3443,east,east_719_166,POLYGON ((-89.28947084013322 41.22729470219819...
4,EPSG:3443,east,east_719_161,POLYGON ((-89.28783801753076 41.11488619546482...


In [19]:
las = gpd.read_file(LAS_TINDEX)
las['location'] = las['location'].str.replace('/mnt/nrcs/isgs/lidar', '/home/jovyan/lidar')
sindex = las.sindex

In [6]:
print(tile.bounds, box(*tile.bounds).bounds)

(-88.68163101887671, 38.75690348028876, -88.5374464825023, 38.86971246863288) (-88.68163101887671, 38.75690348028876, -88.5374464825023, 38.86971246863288)


In [7]:
tile = dems.loc[200, 'geometry']
overlap = las.iloc[list(sindex.intersection(tile.bounds))]


In [30]:

def raster_merge(fname, flist, preserve=True):
    
    for f in flist:
        if not os.path.exists(f):
            print(f'{f} not found')
            return None
        
    wbt = WhiteboxTools()
    wbt.verbose = False
    wbt.mosaic(inputs=';'.join(flist), output=fname, method='nn')
    
    if not preserve:
        for f in flist:
            try:
                os.remove(f)
            except FileNotFoundError:
                pass
    return fname
    
def generate_quads(fname, bounds):
    basename, ext = os.path.splitext(fname)
    minx, miny, maxx, maxy = bounds
    midx, midy = minx+(maxx-minx)/2, miny+(maxy-miny)/2
    return {basename+'_sw'+ext: (minx, miny, midx, midy),
            basename+'_nw'+ext: (minx, midy, midx, maxy),
            basename+'_se'+ext: (midx, miny, maxx, midy),
            basename+'_ne'+ext: (midx, midy, maxx, maxy)}

def merge_tiles(fname, bounds, tiles, sindex, maxfiles=200):
    
    if os.path.exists(fname):
        return
    
    basename, ext = os.path.splitext(fname)
    overlap = las.iloc[list(sindex.intersection(bounds))]
    if len(overlap) < maxfiles:
        raster_merge(fname, [f.replace('.las', ext) for f in overlap['location']])

    else:
        flist = []
        for qname,qbounds in generate_quads(fname, bounds).items():
            flist.append(qname)
            overlap = las.iloc[list(sindex.intersection(qbounds))]
            raster_merge(qname, [f.replace('.las', ext) for f in overlap['location']])
        
        raster_merge(fname, flist, preserve=False)
        
def rec_merge(fname, bounds, tiles, sindex, maxfile=200):
    basename, ext = os.path.splitext(fname)


In [31]:
samples = dems.sample(1)
for fname, bounds in zip(samples['name'], samples.bounds.itertuples()):
    merge_tiles(f'/home/jovyan/{fname}.tif', bounds[1:], las, sindex, maxfiles=200)


In [33]:
x = gpd.overlay(las, dems.iloc[10], how='intersection')

AttributeError: 'Series' object has no attribute 'geom_type'